# Preprocessing

## Installation/import bibliothèques

In [ ]:
!pip install keras-ocr

In [ ]:
!nvidia-smi

In [4]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

import os
import pathlib
import keras_ocr
import math

## Import des données

In [ ]:
!git clone -b dev https://github.com/Finweto/Projet-deep-learning-2022.git
!ls

## Fonction de preprocessing

### Fonction de resize de l'image

In [5]:
def resize_image(img, size=(28,28)):

    h, w = img.shape[:2]
    c = img.shape[2] if len(img.shape)>2 else 1

    if h == w: 
        return cv2.resize(img, size, cv2.INTER_AREA)

    dif = h if h > w else w

    interpolation = cv2.INTER_AREA if dif > (size[0]+size[1])//2 else cv2.INTER_CUBIC

    x_pos = (dif - w)//2
    y_pos = (dif - h)//2

    if len(img.shape) == 2:
        mask = np.zeros((dif, dif), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w] = img[:h, :w]
    else:
        mask = np.zeros((dif, dif, c), dtype=img.dtype)
        mask[y_pos:y_pos+h, x_pos:x_pos+w, :] = img[:h, :w, :]

    return cv2.resize(mask, size, interpolation)

### Fonction de kmeans

In [6]:
def kmeans(img):

  Z = img.reshape((-1,3))
  # convert to np.float32
  Z = np.float32(Z)
  # define criteria, number of clusters(K) and apply kmeans()
  criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

  K = 3

  attemps = 10
  #ret,label,centers=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)
  _,labels,(centers)=cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
  # Now convert back into uint8, and make original image

  centers = np.uint8(centers)
  res = centers[labels.flatten()]
  res2 = res.reshape((img.shape))

  return res2

### Fonction de suppression du texte

In [7]:
def midpoint(x1, y1, x2, y2):
    x_mid = int((x1 + x2)/2)
    y_mid = int((y1 + y2)/2)
    return (x_mid, y_mid)

def inpaint_text(img_path, pipeline):
    # read image
    img = keras_ocr.tools.read(img_path)
    # generate (word, box) tuples 
    prediction_groups = pipeline.recognize([img])
    mask = np.zeros(img.shape[:2], dtype="uint8")
    for box in prediction_groups[0]:
        x0, y0 = box[1][0]
        x1, y1 = box[1][1] 
        x2, y2 = box[1][2]
        x3, y3 = box[1][3] 
        
        x_mid0, y_mid0 = midpoint(x1, y1, x2, y2)
        x_mid1, y_mi1 = midpoint(x0, y0, x3, y3)
        
        thickness = int(math.sqrt( (x2 - x1)**2 + (y2 - y1)**2 ))
        
        cv2.line(mask, (x_mid0, y_mid0), (x_mid1, y_mi1), 255,    
        thickness)
        img = cv2.inpaint(img, mask, 7, cv2.INPAINT_NS)
                 
    return(img)

###Preprocessing des images

Les données préparées sont envoyées dans le dossier dataset.

In [17]:
def preprocess_dataset(dir_path: pathlib.Path):
  os.mkdir('/content/dataset')
  pipeline = keras_ocr.pipeline.Pipeline()
  for subdir_path in dir_path.iterdir():

    new_path = '/content/dataset'
    os.mkdir(new_path+"/"+os.path.basename(subdir_path))

    for plant_dir_path in subdir_path.iterdir():

      new_plant_dir_path = str(new_path)+"/"+os.path.basename(subdir_path)+"/"+os.path.basename(plant_dir_path)
      
      os.mkdir(str(new_plant_dir_path))

      for image_path in plant_dir_path.iterdir():

        image = cv2.imread(str(image_path))
        image = resize_image(image, size=(int(image.shape[0]/4),int(image.shape[1]/4)))

        mask = inpaint_text(image,pipeline)

        img_rgb = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)

        mask = kmeans(img_rgb)
        cv2.imwrite(str(new_plant_dir_path)+"/"+os.path.basename(image_path),mask)

  return "ok"


Lancement du preprocessing

In [ ]:
preprocess_dataset(pathlib.Path('/content/Projet-deep-learning-2022/dataset/'))